In [1]:
!pip install datasets transformers tensorflow numpy matplotlib opencv-python scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.0 MB/s eta 0:00:00


In [12]:
!pip install keras-applications tensorflow_hub

In [2]:
import numpy as np
from PIL import Image
from datasets import load_dataset

# تحميل مجموعة البيانات
dataset = load_dataset("yasimed/lung_cancer")

# دالة استخراج التصنيفات من النصوص
def extract_label(text):
    return 1 if "cancer" in text.lower() else 0  # تحويل النص إلى تصنيف رقمي

# استخراج الصور والتصنيفات مع التأكد من الاتساق
images = []
labels = []

for item in dataset['train']:
    # تحويل الصورة إلى كائن PIL
    image = item['image'].convert("RGB")  # تحويل الصورة إلى 3 قنوات (RGB)

    # تغيير الحجم إلى 224x224
    image = image.resize((224, 224))

    # تحويل الصورة إلى مصفوفة NumPy وتطبيع القيم بين 0 و 1
    image = np.array(image, dtype=np.float32) / 255.0

    images.append(image)
    labels.append(extract_label(item['text']))  # استخراج التصنيف من النص

# تحويل القوائم إلى مصفوفات NumPy بعد التأكد من التجانس
images = np.array(images, dtype=np.float32)
labels = np.array(labels, dtype=np.int32)

# التحقق من الأشكال
print("✅ شكل الصور:", images.shape)  # يجب أن يكون (613, 224, 224, 3)
print("✅ شكل التصنيفات:", labels.shape)  # يجب أن يكون (613,)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/313 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/59.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/613 [00:00<?, ? examples/s]

✅ شكل الصور: (613, 224, 224, 3)
✅ شكل التصنيفات: (613,)


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np


In [4]:
from sklearn.model_selection import train_test_split

# تقسيم البيانات إلى تدريب واختبار
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# طباعة حجم البيانات بعد التقسيم
print("✅ عدد صور التدريب:", X_train.shape[0])
print("✅ عدد صور الاختبار:", X_test.shape[0])


✅ عدد صور التدريب: 490
✅ عدد صور الاختبار: 123


In [5]:
# إنشاء محول بيانات التدريب
train_datagen = ImageDataGenerator(
    rotation_range=20,  # تدوير عشوائي حتى 20 درجة
    width_shift_range=0.1,  # إزاحة أفقية بنسبة 10%
    height_shift_range=0.1,  # إزاحة رأسية بنسبة 10%
    horizontal_flip=True,  # قلب الصورة أفقيًا
)

# تجهيز البيانات للتدريب والاختبار
train_generator = train_datagen.flow(X_train, y_train, batch_size=32, shuffle=True)
test_generator = ImageDataGenerator().flow(X_test, y_test, batch_size=32, shuffle=False)


In [17]:
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

# تحميل ResNet101 بدون الطبقات العليا (include_top=False)
base_model = ResNet101(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# تجميد الطبقات الأصلية حتى لا يتم تعديل أوزانها أثناء التدريب
base_model.trainable = False

# إضافة الطبقات المخصصة
x = GlobalAveragePooling2D()(base_model.output)  # تقليل الأبعاد
x = Dense(256, activation='relu')(x)  # طبقة مخفية مع تنشيط ReLU
x = Dropout(0.5)(x)  # تقليل فرط التكيف (Overfitting)
output = Dense(1, activation='sigmoid')(x)  # مخرجات ثنائية (سرطان أو لا)

# إنشاء النموذج
model = Model(inputs=base_model.input, outputs=output)

# تجميع النموذج
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# عرض ملخص النموذج
model.summary()

171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 43,182,977 (164.73 MB)

 Trainable params: 524,801 (2.00 MB)

 Non-trainable params: 42,658,176 (162.73 MB)

In [18]:
# تدريب النموذج
history = model.fit(train_generator, validation_data=test_generator, epochs=40)

# حفظ النموذج بعد التدريب
model.save("ResNeXt101.h5")


Epoch 1/40


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.7796 - loss: 0.5279 - val_accuracy: 0.9593 - val_loss: 0.2541
Epoch 2/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 295ms/step - accuracy: 0.8990 - loss: 0.2598 - val_accuracy: 0.9593 - val_loss: 0.2060
Epoch 3/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 286ms/step - accuracy: 0.8854 - loss: 0.2956 - val_accuracy: 0.9593 - val_loss: 0.1671
Epoch 4/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 313ms/step - accuracy: 0.9152 - loss: 0.2307 - val_accuracy: 0.9593 - val_loss: 0.1058
Epoch 5/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 273ms/step - accuracy: 0.9378 - loss: 0.1689 - val_accuracy: 0.9593 - val_loss: 0.1036
Epoch 6/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 368ms/step - accuracy: 0.8868 - loss: 0.2422 - val_accuracy: 0.9593 - val_loss: 0.1240
Epoch 7/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 294ms/step - accuracy: 0.9040 - loss: 0.2117 - val_accuracy: 0.9593 - val_loss: 0.1089
Epoch 8/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 282ms/step - accuracy: 0.9231 - loss: 0.1760 - val_accuracy: 0.9431 - val_l

In [19]:
# تقييم النموذج
loss, accuracy = model.evaluate(test_generator)
print(f"✅ دقة النموذج على بيانات الاختبار: {accuracy * 100:.2f}%")


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step - accuracy: 0.9327 - loss: 0.1674
✅ دقة النموذج على بيانات الاختبار: 95.93%


In [ ]:
import numpy as np
from PIL import Image

def preprocess_image(image_path):
    """ تحميل الصورة وتحويلها إلى الشكل المناسب للنموذج """
    image = Image.open(image_path).convert("RGB")  # تحويل إلى 3 قنوات (RGB)
    image = image.resize((224, 224))  # تغيير الحجم إلى 224x224
    image = np.array(image, dtype=np.float32) / 255.0  # تطبيع القيم بين 0 و 1
    image = np.expand_dims(image, axis=0)  # إضافة بعد جديد ليتناسب مع الإدخال المتوقع (batch_size, 224, 224, 3)
    return image

# تجربة الصورة
image_path = "/content/image (5).png"  # ضع مسار الصورة هنا
processed_image = preprocess_image(image_path)

# التنبؤ باستخدام النموذج
prediction = model.predict(processed_image)

# تفسير النتيجة بناءً على النص الموجود في البيانات
def interpret_prediction(pred):
    """ تحويل التنبؤ الرقمي إلى نص بشري """
    if pred >= 0.5:
        return "مصاب بسرطان الرئة 🩸"
    else:
        return "غير مصاب بالسرطان ✅"

# عرض النتيجة النصية مع احتمالية الإصابة
result = interpret_prediction(prediction[0][0])
print(f"🔍 النتيجة: {result} (احتمال الإصابة: {prediction[0][0] * 100:.2f}%)")
